In [ ]:
import requests

def categorize_query_with_gemma(question, model="gemma3:latest"):
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}

    # Structured prompt
    prompt = (
        f"Given the question: \"{question}\"\n"
        "Classify it into a broad category and generate 2–4 relevant tags.\n"
        "Respond only in this format:\n"
        "Category: <Category Name>\n"
        "Tags: <Comma-separated tags>\n"
    )

    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        print(f"[ERROR] Status: {response.status_code}")
        print(response.text)
        return None

# Test it!
question = "What is the famous dish sinigang in the Philippines?"   
result = categorize_query_with_gemma(question)
print(result)


Category: Cuisine
Tags: Sinigang, Filipino Food, Sour Soup, Tamarind
